In [1]:
using Graphs
using SimpleWeightedGraphs
using CSV, DataFrames

In [2]:
network = "crossring/"
folder = network * "l3-c5-d2-bt2_test"
n_tsline = 3

3

In [93]:
TS_data = CSV.read( network * "crossring-network.csv", DataFrame, header = true)

Row,x,y,line,transfer
,Float64,Float64,Int64,String7?
1,-6.0,0.0,1,missing
2,-3.0,0.0,1,"11,19"
3,0.0,0.0,1,8
4,3.0,0.0,1,15
5,6.0,0.0,1,
6,0.0,6.0,2,
7,0.0,3.0,2,13
8,0.0,0.0,2,3
9,0.0,-3.0,2,17


In [94]:
# Function to parse strings into appropriate data types
function parse_column(str)
    if ismissing(str) || isempty(str)
        return Int[]
    elseif occursin(',', str)
        return parse.(Int, split(str, ','))
    else
        return [parse(Int, str)]
    end
end

TS_data.transfer = parse_column.(TS_data.transfer)

19-element Vector{Vector{Int64}}:
 []
 [11, 19]
 [8]
 [15]
 []
 []
 [13]
 [3]
 [17]
 []
 [2]
 []
 [7]
 []
 [4]
 []
 [9]
 []
 [2]

In [8]:
timetables = [CSV.read("$folder/timetable_line$i.csv", DataFrame) for i in 1:n_tsline]

3-element Vector{DataFrame}:
 3×6 DataFrame
 Row │ 1        2        3        4        5        Direction 
     │ Float64  Float64  Float64  Float64  Float64  Float64   
─────┼────────────────────────────────────────────────────────
   1 │    20.0     26.0     32.0     38.0     44.0        1.0
   2 │    49.0     43.0     37.0     31.0     25.0        0.0
   3 │    50.0     56.0     62.0     68.0     74.0        1.0
 2×6 DataFrame
 Row │ 6        7        8        9        10       Direction 
     │ Float64  Float64  Float64  Float64  Float64  Float64   
─────┼────────────────────────────────────────────────────────
   1 │    23.0     29.0     35.0     41.0     47.0        1.0
   2 │    52.0     46.0     40.0     34.0     28.0        0.0
 2×10 DataFrame
 Row │ 11       12       13       14       15       16       17       18       ⋯
     │ Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  ⋯
─────┼─────────────────────────────────────────────────────────────────────

In [96]:
n_layer = maximum(nrow.(timetables))
n_ts = size(TS_data)[1]

19

In [10]:
struct TransitStation
    ID::Int64 # the phsical number of transit stop
    coord::Tuple{Float64,Float64}
    layer_no::Int64
    dep_time::Float64
    arr_time::Float64
    lineNo::Int64 # at which transit line
    transfer::Vector{Int64}
    μ::Float64 # service time
end

In [8]:
# Base.isnan(ts::TransitStation) = !isa(ts, TransitStation)

In [97]:
ts_map = permutedims(reshape(1:n_ts*n_layer, n_ts, n_layer)) # row: layer number; column: transit stops

3×19 Matrix{Int64}:
  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
 20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38
 39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57

In [98]:
dwel_time = 1
ts_nodes = Vector(undef, n_ts*n_layer)
for lyr in 1:n_layer
    for ts in 1:n_ts
        coord = (TS_data[ts,:x], TS_data[ts,:y])
        lineNo = TS_data[ts,:line]
        transfer = TS_data[ts, :transfer]
        if lyr <= size(timetables[lineNo])[1] 
            dep_time = timetables[lineNo][lyr,"$ts"]
            arr_time = dep_time .- dwel_time
            ts_dum = ts_map[lyr, ts]
            ts_nodes[ts_dum] = TransitStation(ts, coord, lyr, dep_time, arr_time, lineNo, transfer, 0.0)
        else
            ts_dum = ts_map[lyr, ts]
            ts_map[lyr, ts] = 0
            ts_nodes[ts_dum] = NaN
        end
    end
end

In [13]:
# filter!(x -> isa(x,TransitStation), ts_nodes)
ts_nodes

57-element Vector{Any}:
    TransitStation(1, (-6.0, 0.0), 1, 20.0, 19.0, 1, [0], 0.0)
    TransitStation(2, (-3.0, 0.0), 1, 26.0, 25.0, 1, [11, 19], 0.0)
    TransitStation(3, (0.0, 0.0), 1, 32.0, 31.0, 1, [8], 0.0)
    TransitStation(4, (3.0, 0.0), 1, 38.0, 37.0, 1, [15], 0.0)
    TransitStation(5, (6.0, 0.0), 1, 44.0, 43.0, 1, [0], 0.0)
    TransitStation(6, (0.0, 6.0), 1, 23.0, 22.0, 2, [0], 0.0)
    TransitStation(7, (0.0, 3.0), 1, 29.0, 28.0, 2, [13], 0.0)
    TransitStation(8, (0.0, 0.0), 1, 35.0, 34.0, 2, [3], 0.0)
    TransitStation(9, (0.0, -3.0), 1, 41.0, 40.0, 2, [17], 0.0)
    TransitStation(10, (0.0, -6.0), 1, 47.0, 46.0, 2, [0], 0.0)
   ⋮
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

In [14]:
ts_graph = SimpleWeightedDiGraph(n_ts*n_layer)

{57, 0} directed simple Int64 graph with Float64 weights

In [43]:
import Base.+
+(range::UnitRange{Int64}, offset::Int64) = UnitRange(range.start + offset, range.stop + offset)

+ (generic function with 218 methods)

In [47]:
stops_line = [count(x -> x == i, TS_data.line) for i in 1:n_tsline]
start_indices = cumsum([1;stops_line[1:end-1]])
ranges_list = [start: start + length - 1 for (start, length) in zip(start_indices, stops_line)]

1:5

In [62]:
ts_arcs = []
ts_arcs_dict = Dict((l,lyr) =>[] for lyr in 1:n_layer for l in 1:n_ts)
for lyr in 1:n_layer
    # for l in 1:n_tsline
    for l in 1:1
        # number_stops = stops_line[l]
        ts_range = ranges_list[l] + 19*(lyr-1)
        if ts_map[lyr,ranges_list[l].start] == 0
            continue
        end
        direction = timetables[l].Direction[lyr]
        println("ts range: ", ts_range)
        
        # arcs within the same line
        for ts1 in ts_range
            for ts2 in ts_range             
                if direction == 1 && ts2 > ts1
                    push!(ts_arcs, (ts1,ts2))
                    push!(ts_arcs_dict[(l,lyr)], (ts1,ts2))
                elseif direction == 0 && ts1 < ts2
                    push!(ts_arcs, (ts2, ts1))
                    push!(ts_arcs_dict[(l,lyr)], (ts2,ts1))
                end
            end
            if isempty(ts1.transfer)
                continue
            else
                
            end
        end

    end

    # if lyr == 1
    #     break
    # end
end

ts range: 1:5
ts range: 20:24
ts range: 39:43


In [115]:
function find_transfer_arcs(ts::Int64, ts_nodes::Vector, ts_map::Matrix)
    ts1 = ts_nodes[ts]
    transfers = ts1.transfer
    for ts2_ID in transfers
        ts2_dummies = ts_map[:,ts2_ID]
        filter!(x -> x == 0, ts2_dummies)
        for ts2_dm in ts2_dummies
            @info ts2_dm
            ts2 = ts_nodes[ts2_dm]
            if ts1.arr_time > ts2.dep_time && ts1.arr_time <= ts2.dep_time + 10
                println(ts1,ts2)
            end
        end
    end
end

find_transfer_arcs (generic function with 3 methods)

In [116]:
find_transfer_arcs(2, ts_nodes, ts_map)

┌ Info: 11
└ @ Main c:\Users\yumeng\OneDrive - University of Luxembourg\Documents\Yumeng\PhDproject\WP2\code\EIDARPgenerator\try_dijkstra.ipynb:7
┌ Info: 30
└ @ Main c:\Users\yumeng\OneDrive - University of Luxembourg\Documents\Yumeng\PhDproject\WP2\code\EIDARPgenerator\try_dijkstra.ipynb:7
┌ Info: 0
└ @ Main c:\Users\yumeng\OneDrive - University of Luxembourg\Documents\Yumeng\PhDproject\WP2\code\EIDARPgenerator\try_dijkstra.ipynb:7


BoundsError: BoundsError: attempt to access 57-element Vector{Any} at index [0]

In [112]:
ts_nodes[2].transfer

2-element Vector{Int64}:
 11
 19